In [49]:
import numpy as np
import pandas as pd

In [62]:
TARGET_COL = 'scalar_coupling_constant'
N_FOLDS = 4
VERSION = 1

In [51]:
PATH = '../tmp/'
SUB_PATH = '../submissions/'
OOF_PATH = '../oofs/'

In [52]:
train_df = pd.read_csv(PATH+'train_proc_df.csv', index_col=0)
val_idxs = pd.read_csv(PATH+'val_idxs_4_fold_cv.csv', index_col=0)

/anaconda2/envs/python36/lib/python3.6/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [54]:
sub_files = [f'mpnn-v1-idx{i}-submission' for i in range(1, N_FOLDS+1)]
oof_files = [f'mpnn-v1-idx{i}-oof' for i in range(1, N_FOLDS+1)]
preds = pd.concat([pd.read_csv(SUB_PATH+f+'.csv', index_col=0) for f in sub_files], axis=1)
oofs = pd.concat([pd.read_csv(OOF_PATH+f+'.csv', index_col=0) for f in oof_files], axis=1)

/anaconda2/envs/python36/lib/python3.6/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [55]:
def group_mean_log_mae(y_true, y_pred, types):
    maes = (y_true - y_pred).abs().groupby(types).mean()
    return np.log(maes).mean()

In [56]:
oof = pd.DataFrame(train_df['molecule_id'])
oof[TARGET_COL] = np.nan
for i in range(N_FOLDS):
    idx = oof['molecule_id'].isin(val_idxs.iloc[:,i].dropna().astype(int))
    oof[TARGET_COL][idx] = oofs.iloc[:,i].dropna().values

group_mean_log_mae(train_df[TARGET_COL], oof[TARGET_COL], train_df['type'])

/anaconda2/envs/python36/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [59]:
sub_ens = pd.DataFrame(preds.mean(axis=1), columns=[TARGET_COL])
sub_ens.head()

,scalar_coupling_constant
id,
4658147,11.409493
4658148,187.245698
4658149,0.203321
4658150,187.245700
4658151,11.409493


In [60]:
sub_ens.to_csv(f'{SUB_PATH}mpnn-v{VERSION}-submission.csv')